<a href="https://colab.research.google.com/github/DSI-Capstone-FS9/ABCD-MRI/blob/master/Pre_Trained_Densenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install tensorflow-gpu

import numpy as np
import pandas as pd
import warnings 
import nibabel as nib
import tensorflow as tf
import os
import math
import glob
import cv2

warnings.simplefilter('ignore')

In [29]:
tf.__version__

'2.0.0'

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
t1_path = '/content/drive/My Drive/Capstone/05_Data/02_Sample_MRI/subsample/t1/'
t1_files = glob.glob(t1_path + '*.gz',recursive=True)
t1_all_files = sorted(t1_files)

t2_path = '/content/drive/My Drive/Capstone/05_Data/02_Sample_MRI/subsample/t2/'
t2_files = glob.glob(t2_path + '*.gz',recursive=True)
t2_all_files = sorted(t2_files)

In [0]:
t1 = pd.DataFrame([t1_all_files]).T
t1['label'] = 0
t1.columns = ['image','label']

t2 = pd.DataFrame([t2_all_files]).T
t2['label'] = 1
t2.columns = ['image','label']

df = pd.concat([t1,t2])

In [0]:
# dimensions to consider for the images
img_rows, img_cols, img_channels = 64,64,64 #64,64,3

# batch size for training  
batch_size=1

# total number of classes in the dataset
nb_classes=2

In [0]:
def data_generator(data, 
                   batch_size,
                   preprocessing_fn = None,
                   is_validation_data=False):
    # Get total number of samples in the data
    n = len(data)
    nb_batches = int(np.ceil(n/batch_size))

    # Get a numpy array of all the indices of the input data
    indices = np.arange(n)
    
    while True:
        if not is_validation_data:
            # shuffle indices for the training data
            np.random.shuffle(indices)
            
        for i in range(nb_batches):
            # get the next batch 
            next_batch_indices = indices[i*batch_size:(i+1)*batch_size]
            nb_examples = len(next_batch_indices)
            
            # Define two numpy arrays for containing batch data and labels
            batch_data = np.zeros((nb_examples, 
                           img_rows, 
                           img_cols, 
                           img_channels), 
                          dtype=np.float32)
            batch_labels = np.zeros((nb_examples, nb_classes), dtype=np.float32)
            
            # process the next batch
            for j, idx in enumerate(next_batch_indices):
                #img = cv2.imread(data.iloc[idx]["image"])
                #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                img = nib.load(data.iloc[idx]["image"])
                img = np.array(img.dataobj)#[:, :, 31:34]
                
                label = data.iloc[idx]["label"]
                
                #if not is_validation_data:
                    #img = seq.augment_image(img)
                
                img = cv2.resize(img, (img_rows, img_cols)).astype(np.float32)
                batch_data[j] = img
                batch_labels[j] = to_categorical(label,num_classes=nb_classes)
            
            if preprocessing_fn is not None:
                batch_data = preprocessing_fn(batch_data)
            
            yield batch_data, batch_labels

In [0]:
#training data generator 
train_data_gen = data_generator(df, 
                                batch_size, 
                                preprocessing_fn=None)

# validation data generator 
valid_data_gen = data_generator(df, 
                                batch_size,
                                preprocessing_fn=None,
                                is_validation_data=True)

In [36]:
from skimage.io import imread, imsave
from skimage.transform import resize 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras import layers as L
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

import tensorflow as tf
import tensorflow.keras.backend as K

import imgaug as ia
from imgaug import augmenters as iaa

def get_base_model():
    base_model = DenseNet121(input_shape=(img_rows, img_cols, img_channels), 
                       weights=None, 
                       include_top=False)
    return base_model

# get the base model
base_model = get_base_model()

#  get the output of the second last dense layer 
base_model_output = base_model.layers[-2].output

# add new layers 
x = L.Dropout(0.5,name='drop2')(base_model_output)
output = L.Dense(nb_classes, activation='softmax', name='fc3')(x)
output = L.Flatten()(output)
output = L.Dense((2), activation='softmax', name='fc4')(output)

# define a new model 
model = Model(base_model.input, output)

# Freeze all the base model layers 
for layer in base_model.layers[:-1]:
    layer.trainable=False

# compile the model and check it 
optimizer = RMSprop(0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 64, 64, 64)] 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 70, 70, 64)   0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 32, 32, 64)   200704      zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [0]:
# Use earlystopping 
es = EarlyStopping(patience=3, restore_best_weights=True)

# checkpoint to save model
chkpt = ModelCheckpoint(filepath="monkey_model.h5", save_best_only=True)

# number of training and validation steps for training and validation
nb_train_steps = int(np.ceil(len(df)/batch_size))
nb_valid_steps = int(np.ceil(len(df)/batch_size))

# number of epochs 
nb_epochs=10

In [38]:
%%time
# train the model 
history1 = model.fit_generator(train_data_gen, 
                              epochs=nb_epochs, 
                              steps_per_epoch=nb_train_steps, 
                              validation_data=valid_data_gen, 
                              validation_steps=nb_valid_steps,
                              callbacks=[es,chkpt])

Epoch 1/10
60/60 [==============================] - 51s 855ms/step - loss: 0.9318 - accuracy: 0.5000 - val_loss: 0.7729 - val_accuracy: 0.5000
Epoch 2/10
60/60 [==============================] - 52s 865ms/step - loss: 0.7903 - accuracy: 0.5000 - val_loss: 0.7353 - val_accuracy: 0.5000
Epoch 3/10
60/60 [==============================] - 53s 880ms/step - loss: 0.7292 - accuracy: 0.5000 - val_loss: 0.7054 - val_accuracy: 0.5000
Epoch 4/10
60/60 [==============================] - 51s 855ms/step - loss: 0.6967 - accuracy: 0.5000 - val_loss: 0.6823 - val_accuracy: 0.5000
Epoch 5/10
60/60 [==============================] - 51s 858ms/step - loss: 0.6527 - accuracy: 0.5000 - val_loss: 0.6818 - val_accuracy: 0.5000
Epoch 6/10
60/60 [==============================] - 50s 840ms/step - loss: 0.6038 - accuracy: 0.5500 - val_loss: 0.7420 - val_accuracy: 0.5167
Epoch 7/10
60/60 [==============================] - 52s 860ms/step - loss: 0.5556 - accuracy: 0.7833 - val_loss: 0.6977 - val_accuracy: 0.5000